# LightGBM Regressor 

## Mục tiêu:
- Xây dựng mô hình LightGBM Regressor để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation
- Đánh giá mô hình trên tập validation và test
- Lưu mô hình đã huấn luyện 

## Giới thiệu

LightGBM (Light Gradient Boosting Machine) là framework gradient boosting được phát triển bởi Microsoft, tối ưu hóa cho tốc độ và hiệu suất cao.

## Ưu điểm

- Tốc độ huấn luyện cực nhanh (nhanh hơn các GBM khác).

- Hiệu suất cao với dữ liệu lớn.

- Xử lý tốt dữ liệu nhiều chiều.

## Nhược điểm

- Dễ bị overfitting với dữ liệu nhỏ.

- Nhạy cảm với siêu tham số.

## Bước 1 - Import các thư viện cần thiết

In [ ]:
# Thư viện cơ bản
import numpy as np
import pandas as pd
import os

# Thư viện LightGBM
from lightgbm import LGBMRegressor

# Thư viện sklearn cho mô hình và đánh giá
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

## Bước 2 - Đọc dữ liệu đã tiền xử lý

Đọc dữ liệu từ các file CSV đã được tiền xử lý và chia sẵn thành train/validation/test.

In [ ]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (đã được chuẩn hóa):")
train_df.head()

## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`.

In [ ]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

## Bước 4 - Xây dựng và huấn luyện mô hình LightGBM

4.1. Sử dụng GridSearchCV với 5-Fold Cross-Validation để tìm kiếm siêu tham số tối ưu cho mô hình LightGBM.

In [ ]:
# Định nghĩa không gian siêu tham số để tìm kiếm
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, -1],
    'num_leaves': [15, 31, 63],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.8, 0.9, 1.0]
}

print(f"Số lượng tổ hợp tham số: {np.prod([len(v) for v in param_grid.values()])}")
print("Phương pháp: GridSearchCV với 5-Fold Cross-Validation")
print("Metric đánh giá: Negative RMSE (neg_root_mean_squared_error)")
print("\nĐang thực hiện tìm kiếm...")
print("="*60)

# Tạo GridSearchCV
grid_search = GridSearchCV(
    estimator=LGBMRegressor(random_state=42, verbose=-1),
    param_grid=param_grid,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1,
)

# Thực hiện tìm kiếm
grid_search.fit(X_train, y_train)

4.2. Hiển thị siêu tham số tối ưu và đánh giá hiệu suất của mô hình tốt nhất.

In [ ]:
# Lấy mô hình tốt nhất và tham số tối ưu
best_lgbm = grid_search.best_estimator_
best_params = grid_search.best_params_

print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params.items():
    print(f"  {param:20s}: {value}")

# Thông tin về mô hình tối ưu
print(f"\nSố lượng cây: {best_lgbm.n_estimators}")
print(f"Learning rate: {best_lgbm.learning_rate}")

4.3 Đánh giá mô hình tối ưu trên tập train

In [ ]:
y_pred = best_lgbm.predict(X_train)

# Tính toán các metrics cho tập train
train_mae = mean_absolute_error(y_train, y_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
train_r2 = r2_score(y_train, y_pred)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: LightGBM:")
print(f"RMSE loss: {train_rmse:.3f}")
print(f"MAE loss: {train_mae:.3f}")
print(f"R2 loss: {train_r2:.3f}")

4.4. Lưu mô hình LightGBM đã tối ưu vào file để sử dụng sau này.

In [ ]:
# Tạo thư mục saved_models nếu chưa có
os.makedirs('../model/6_lightGBM', exist_ok=True)

# Lưu mô hình
model_path = '../model/6_lightGBM/lightgbm.pkl'
joblib.dump(best_lgbm, model_path)

## Kết luận

### Tổng kết:
1. Mô hình LightGBM đã được xây dựng và tối ưu hóa thành công
2. GridSearchCV với 5-Fold CV đã tìm được bộ siêu tham số tối ưu
3. Mô hình được đánh giá dựa trên 3 metrics: MAE, RMSE và R2